# Моделирование многочастичной системы
## Написание программы

In [1]:
import numpy as np 

from sklearn.neighbors import NearestNeighbors

import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from matplotlib import animation
matplotlib.use('Qt5Agg')

from tqdm import tqdm
import scipy.stats as stats
import math
import warnings
warnings.filterwarnings("ignore")
import time

### Основные глобальные параметры системы

Будем измерять величины слудующим образом. Отталкиваемся от формулы для потенциала Леннарда-Джонса $U=4E(\frac{\sigma^{12}}{r^{12}}-\frac{\sigma^6}{r^6})$. Величины без звездочки измеряются в СИ:




$r=r^* \sigma  ; t=\sigma \sqrt{\frac{m}{E}} t^*$


$T=\frac{E}{k_b}T^* ; V=V^* \sqrt{\frac{E}{m}}$


$U^*(r^*)=\frac{U(r^*)}{E}$


Для каждого газа $\sigma$ и $\frac{E}{k_b}$ разные



Посчитаем нормировные коэффициенты

In [2]:
sigma=4.010*10**(-10)  #метры  Данные для молекулы О2
E_kb=205                #кельвины (E/kb)
mol_mass=5.33*10**(-26)      #масса 1 молекулы в кг
kb=1.380649*10**(-23) #постоянная Больцмана Дж/к
sec=sigma*(mol_mass/(E_kb*kb))**(1/2) #нормировка по времени

Введите основные данные системы в СИ

In [3]:
aa, bb =12,6               #степени потенциала Леннарда-Джонса
dimension=2                   #размерность пространства(пока работает только с 2 и 3)


R_particles=10**(-10)  #радиус молекул в метрах
temp=300   #температура кельвины

steps=int(5*10**4)  # колво шагов итераций
tmax=1e-10     #время в секундах. Перед тем как вводить посмотрите, какая нормировка на время, там очень маленькая степень 

Произведем нормировку

In [4]:
R_particles/=sigma
temp/=E_kb
tmax/=sec
dt=tmax/float(steps)
print('Характерное время: {}\nХарактерная температура: {}\nХарактерный радиус молекул: {}'.format(tmax,temp,R_particles))
print('Шаг по времене = {0:.5f}'.format(dt))

Характерное время: 57.465950540361774
Характерная температура: 1.4634146341463414
Характерный радиус молекул: 0.24937655860349128
Шаг по времене = 0.00115


### Генерация начальных положений и скоростей частиц 

Сгенерируем частицы в объеме параллепипеда и шара. Сделаем проверку, частицы не должны располагаться к ограниченной области и друг-другу ближе, чем их радиусы.

Система замнута, поэтому зануленная скорость центра масс должна сохранятся со временем.

In [5]:
def generate(generation_step=1): #a_box,b_box,c_box,box_form -глобальные переменные, стороны и форма бокса
    """
    Функция, генерирующая начальное положение и скорости частиц. 
    
    Зависит от множества глобальных параметров: dimension, count_of_particles, temp, R_particles, box_form и box_size.
    Выводит матрицы начальных координат и скоростей, размерности которых равны (count_of_particles, dimension).

    !!!Внимательно следите за соотношением размера бокса и количества частиц. В программе существует начальное расстояние между генерацией частиц, 
    чтобы избежать сил больших порядков и, как следствие, быстрого расхождения алгоритма.
    """
   # сгенерируем нормированные скорости V*, как перейти в СИ написано выше
    v=np.float64(np.random.uniform(-0.5,0.5,(count_of_particles,dimension)))   #равномерное распределение скоростей от -0,5 до 0,5 в проекциях x,y,z (part,3)
    sumv=np.float64(np.sum(v,axis=0)/count_of_particles)     #скорость центра масс по x,y,z (1,3)
    sumv2=np.float64(np.sum(v**2)/count_of_particles)        #средний квадрат скорости  (1)
    fs=np.float64((dimension*temp/sumv2)**(1/2))                    # коэффициент перемаштабирования скоростей (1)
    v=(v-sumv)*fs                                #Делаем так, чтобы занулить скорость центра масс системы (part,3)
    sumv=np.sum(v,axis=0)/count_of_particles     #скорость центра масс по x,y,z (1,3)
    sumv2=np.sum(v**2)/count_of_particles 
    
    if box_form =='parallelepiped':
        a=box_size      # стороны параллепипеда a,b,c !!!посмотреть нормировку Радиус молекул где-то 0.2-1
        if dimension==2: a=a[:-1]
        a=(a-2*R_particles)   #сделаем маленький оступ от стенок
        x=np.float64(np.random.uniform(-a/2,a/2,(count_of_particles,dimension))) # матрица координат частиц ()
        
        for i in range(count_of_particles): #такая сумма в любом случае будет не меньше 1, тк расстояния частицы до ее самой равно 0=>1 True точно будет
            while sum( np.sum(((x-x[i])**2),axis=1)<=((2*R_particles+generation_step)**2)) >1:  #Считаем расстояния между i-той частицей и всеми остальными,
                 x[i]=np.random.uniform(-a/2,a/2,dimension)#если оно меньше чем радиус, то рандомим еще раз 
                                                                                                     


    return x,v # НАЧАЛЬНЫЕ КООРДИНАТЫ И скорости



### Взаимодействие частиц
#### Потенциал Леннарда-Джонса. Упругое соударение с боксом

$U(r)=c_{\alpha \beta}(\frac{1}{r^{\alpha}}-\frac{1}{r^\beta})$

Вычислим x-составляюущую силы

$f_x=-\frac{x}{r}\frac{\partial U(r)}{\partial r}$ 

Для леннард-джонсовской системы 

$f_x=\frac{c_{\alpha \beta}*x}{r^2}(\frac{\alpha}{r^{\alpha}}-\frac{\beta}{r^{\beta}})$

Коэффициент $c_{\alpha \beta}$ находится из $U_{min}(c_{\alpha \beta},r_{o})=-1$


$r_o$- точка, в которой реализуется минимум потенциала

In [6]:
def c_aa_bb(aa,bb):
   if aa!=bb and aa!=0 and bb!=0:
       r0=(aa/bb)**(1/(aa-bb))
       return -1/(1/r0**aa-1/r0**bb)
   else:
       r0=(aa)**(1/(aa))
       return 1/(1/r0**aa)


Caabb=c_aa_bb(12,6)
Caabb

4.0

In [7]:
def LJForsesPRO(x):
    nbrs = NearestNeighbors(n_neighbors=n_neighborsLJ, algorithm='ball_tree').fit(x)
    distances, indices = nbrs.kneighbors(x)
    LJF = np.zeros((count_of_particles,dimension))
    ff = Caabb/distances[:,1:]**2*(aa/distances[:,1:]**aa-bb/distances[:,1:]**bb) # Матрица ( (count_of_particlles, n_neighbors-1) )
    for i in range(count_of_particles):
        x_xi = -x[indices[i,1:]]+x[i] # ( (n_neighbors-1,3))
        LJF[i] = np.sum(ff[i,None].T*x_xi, axis = 0)  # ( (1,3))
    return LJF

Учтем упругие соударения с боксом

In [8]:
def bounce_box(x,v):
    for i in range(dimension):
        rightside=x[:,i]>(box_size[i]/2-R_particles)
        leftside=x[:,i]<(-box_size[i]/2+R_particles)
        if any(rightside):
            x[rightside, i] = -x[rightside, i]+2*(box_size[i]/2-R_particles)
            v[rightside, i] = -v[rightside, i]
        if any(leftside):
            x[leftside, i] = -x[leftside, i]+2*(-box_size[i]/2+R_particles)
            v[leftside, i] = -v[leftside, i]
    return x,v

#### В книг Брилиантова 19 страница можно найти формулы для скоростей соударающихся молекул


$\vec{V^{'}_{1}} = \vec{V_1} - \frac{1}{2} (1+E)(\vec{V_{1}},\vec{e})\vec{e}$


$\vec{V^{'}_{2}} = \vec{V_2} + \frac{1}{2} (1+E)(\vec{V_{1}},\vec{e})\vec{e}$


$\vec{e}=\frac{\vec{r}_{1-2}}{|\vec{r}_{1-2}|}$


$\vec{V}_{1-2} = \vec{V}_1 - \vec{V}_2$

In [9]:
def coef_restitution(v_scalar): #Проверить формулу (Можно аппроксимировать, как coef_rest = 1-C*V^(1/5) см статью 3)
    #Young_modulus = 0.2
    #Poisson_ratio = 0.3
    #A = 0.1
    #alpha = (3/2)**(3/2)*Young_modulus*2*(R_particles/2)**(1/2)/(1-Poisson_ratio**2) # модуль юнга и 
    #c1 = 1.1534
    #c2 = 3/5*c1**2
    return 1-D1*abs(v_scalar)**(1/5)#c1*A*alpha**(2/5)*v_scalar**(1/5)+c2*A**2*alpha**(4/5)*

In [10]:
def collisions(x, v):
    if triple_collisions == True:
            nbrs = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(x)
            distances, indices = nbrs.kneighbors(x)
            indices = indices[(distances[:,1]<=2*R_particles)*(distances[:,2]<=2*R_particles)]
            a = np.array([[0,0,0]])
            for i in range(len(indices)):
                if np.sort(indices[i]) in a: continue
                else: 
                        a=np.vstack((np.sort(indices[i]),a))
                        
                        r_12 = x[indices[i,0]]-x[indices[i,1]]  #столкновение средней и первой
                        e = r_12 / sum(r_12**2)**(1/2)
                        v_12 = v[indices[i,0]]-v[indices[i,1]]
                        v_scalar = sum(v_12*e)
                        dr = (2*R_particles-sum(r_12**2)**(1/2))*e
                        x[indices[i,0]]+=dr/2
                        x[indices[i,1]]-=dr/2
                        v[indices[i,0]] = v[indices[i,0]] - 1/2*(1+coef_restitution(v_scalar=v_scalar))*v_scalar*e
                        v[indices[i,1]] = v[indices[i,1]] + 1/2*(1+coef_restitution(v_scalar=v_scalar))*v_scalar*e

                        r_12 = x[indices[i,0]]-x[indices[i,2]] #столкновение средней и второй
                        e = r_12 / sum(r_12**2)**(1/2)
                        v_12 = v[indices[i,0]]-v[indices[i,2]]
                        v_scalar = sum(v_12*e)
                        dr = (2*R_particles-sum(r_12**2)**(1/2))*e
                        x[indices[i,0]]+=dr/2
                        x[indices[i,2]]-=dr/2
                        v[indices[i,0]] = v[indices[i,0]] - 1/2*(1+coef_restitution(v_scalar=v_scalar))*v_scalar*e
                        v[indices[i,2]] = v[indices[i,2]] + 1/2*(1+coef_restitution(v_scalar=v_scalar))*v_scalar*e

                        r_12 = x[indices[i,0]]-x[indices[i,1]] #столкновение средней и первой
                        e = r_12 / sum(r_12**2)**(1/2)
                        v_12 = v[indices[i,0]]-v[indices[i,1]]
                        v_scalar = sum(v_12*e)
                        v[indices[i,0]] = v[indices[i,0]] - 1/2*(1+coef_restitution(v_scalar=v_scalar))*v_scalar*e
                        v[indices[i,1]] = v[indices[i,1]] + 1/2*(1+coef_restitution(v_scalar=v_scalar))*v_scalar*e


    nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(x)
    distances, indices = nbrs.kneighbors(x)
    indices = np.sort(indices[distances[:,1]<=2*R_particles],axis=1)
    for i in range(0,len(indices),2):
            r_12 = x[indices[i,0]]-x[indices[i,1]]
            e = r_12 / sum(r_12**2)**(1/2)
            v_12 = v[indices[i,0]]-v[indices[i,1]]
            v_scalar = sum(v_12*e)
            dr = (2*R_particles-sum(r_12**2)**(1/2))*e
            x[indices[i,0]]+=dr/2
            x[indices[i,1]]-=dr/2
            v[indices[i,0]] = v[indices[i,0]] - 1/2*(1+coef_restitution(v_scalar=v_scalar))*v_scalar*e
            v[indices[i,1]] = v[indices[i,1]] + 1/2*(1+coef_restitution(v_scalar=v_scalar))*v_scalar*e

    return x, v, len(indices)/2

### Итерационный алгоритм
#### Алгоритм Верле

Вывод алгоритма начнем с разложения в ряд Тейлора для координаты частицы в момент времени t(страница 69)


$r(t+\Delta t)=r(t)+V(t)\Delta t +\frac{f(t)}{2m}\Delta t^2$

Скорость:


$V(t+\Delta t)=V(t)+\frac{f(t+\Delta t)+f(t)}{2m}\Delta t$


Данный алгоритм эквивалентен алгоритму Верле в оригинальной форме


In [11]:
def iteration1(x,v): #одна итерация  #возможно нужно поделить на массы
    
    ff=LJForsesPRO(x)
    x=x+v*dt+ff/2*(dt**2)
    ff_new=LJForsesPRO(x)
    v=v+(ff_new+ff)/2*dt

    x, v = bounce_box(x, v)

    #sumv=np.sum(v_new,axis=0)/count_of_particles #скорость центра масс
    #sumv2=np.sum(v_new**2)/count_of_particles   #кенетическая энергия на одну молекулу
    #emp=sumv2/(3*count_of_particles)       #мгновенная температура
    #energy=1/2*sumv2      

    return x,v,0#,temp,energy

#### Случай, если в системе присутствуют только столкновения

In [12]:
def iteration2(x, v):
    x = x+v*dt
    x, v, collisions_per_interation = collisions(x, v)
    x, v = bounce_box(x, v)
    return x, v, collisions_per_interation

### Расстояние Кульбака-Лейблера.


Данное расcтояние позволяет оценить близость двух распределений, нас интересует, непосредственно, близость распределения скоростей и распределения Максвелла.


$KL(P||Q) = \int P(x) log \frac{P(x)}{Q(x)} dx$


$KL(P||Q) = \sum_i P(i) log \frac{P(i)}{Q(i)}$

i - наблюдения

In [13]:
def KLdivergence(mu, sigma, newbar): #передаем параметры распределения с которым хотим сравнить и распределение(высоту баров) распределения скоростей в данный момент
    scales = np.random.normal(mu, sigma, 10**5)
    HIST_BINS = np.linspace(-4,4, len(newbar)+1)
    newbarnormal,_=np.histogram(scales, HIST_BINS, normed=True) # массив высот баров из выборки нормального распределения длиной len(newbar)
    newbarnormal = newbarnormal[newbar>0]
    newbar = newbar[newbar>0]
    return np.sum(newbar*(HIST_BINS[-1]-HIST_BINS[-2])*np.log(newbar/newbarnormal)) # умножение на ширину бара нужно формально (чтобы была вероятность)

### Визуализация.

In [14]:
def animhists(x,v,iteration, ips=10): 
    matplotlib.use('Qt5Agg')    
    time = 0 #время с момента начала анимации, те время движения молекул
    KL = np.zeros(dimension)
    count_of_collisions = 0
    
    plt.ion()

    fig, ax = plt.subplots(nrows=1,ncols=dimension,figsize=(18,10))
    if count_of_particles>200:  bins_count = 30                              # "по-умному" зададим кол-во баров
    elif count_of_particles>=40: bins_count=(count_of_particles)**1/2/np.log2(count_of_particles)*1.6+11
    elif count_of_particles<40: bins_count=(count_of_particles)**1/2/np.log2(count_of_particles)*1.8+9
    HIST_BINS=np.linspace(-4,4,round(bins_count))
    bars=[]
    lines=[]
    KL_history = [[] for i in range(dimension)]

    mu = 0   #скорость центра масс у нас равна 0, поэтому мат ожидание 0
    variance = np.sum(v**2)/count_of_particles
    sigma = math.sqrt(variance/dimension)
    KL_time_history=[[] for i in range(dimension)]
    KL_collisions_history = [[] for i in range(dimension)]
    collisions_history = []

    for i in range(dimension):
        _,_,bar_cont = ax[i].hist(v[:,i],HIST_BINS,density=True)  #строим гистограмму и получаем бары 
        bars.append(bar_cont)
        line,=ax[i].plot(np.linspace(mu - 3*sigma, mu + 3*sigma, 100), stats.norm.pdf(np.linspace(mu - 3*sigma, mu + 3*sigma, 100), mu, sigma))
        lines.append(line)
        ax[i].set_ylim(top=0.6)

    while len(plt.get_fignums()) > 0:
        collisions_per_iteration = 0
        for j in range(ips):
            x, v, c_p_i = iteration(x, v)
            collisions_per_iteration+=c_p_i
        time+=ips*dt

        if collisions_per_iteration !=0:
            count_of_collisions+=collisions_per_iteration
            collisions_history.append(count_of_collisions) 

            
        for i in range(dimension):
            newbar,_=np.histogram(v[:,i],HIST_BINS,normed=True)  #находим новые бары

            mu = np.sum(v[:,dimension])   #скорость центра масс у нас равна 0, поэтому мат ожидание 0
            variance = np.sum(v**2)/count_of_particles  # <v^2>-<v>^2, dimension/2 kT = mv^2 /2, sigma = sqrt(kT/m) => sigma = sqrt(v^2 /dimension)
            sigma = (variance/dimension)**(1/2) 
            lines[i].set_xdata(np.linspace(mu - 3*sigma, mu + 3*sigma, 100))  # из-за того, что потенциальная энергия переходит в кинетическую,успредненный квдарат скорости меняется => меняется распределение
            lines[i].set_ydata( stats.norm.pdf(np.linspace(mu - 3*sigma, mu + 3*sigma, 100), mu, sigma))
        
            KL[i] = KLdivergence(mu, sigma, newbar)
            KL_history[i].append(KL[i])
            
            if collisions_per_iteration !=0: 
                KL_collisions_history[i].append(KL[i])

            ax[i].set_title('{0}Распред. скор. {1}. KL = {2:.3f}'.format([str(count_of_particles)+' частиц. ','',''][i], ['Vx','Vy'+'. Момент вр. t='+str(round(time,2)),'Vz'][i],KL[i]))
            for count, rect in zip(newbar, bars[i].patches):
              rect.set_height(count)                             #меняем старые бары на новые
              
        plt.draw()
        plt.gcf().canvas.flush_events() 

    plt.ioff()
    plt.show()

    return KL_history

In [15]:
def animationND(x, v, iteration, ips=10):
    """
    Функция, анимирующая движение частиц на плоскости. 
    
    Принимает на вход начальные координаты и скорости. Векторы должны быть длины dimesion=2.
    Выводит анимацию движения частиц в боксе и гистограммы распределения их скоростей.
    """
    count_of_collisions = 0
    time = 0 #время с момента начала анимации, те время движения молекул
    KL = np.zeros(dimension)
    plt.ion()
    fig,ax = plt.subplots(nrows=dimension,ncols=3,figsize=(19,10))
    
    gs = ax[0, 0].get_gridspec()    #делаем 1 большое окно и 2 маленьких с гистограммами 
    for i in range(dimension):
        for ax0 in ax[i,:2]: ax0.remove()
    if dimension==3: 
        axbig = fig.add_subplot(gs[:dimension, :2], projection='3d')
        axbig.set(xlim=(-box_size[0]/2, box_size[0]/2), ylim=(-box_size[1]/2, box_size[1]/2), zlim=(-box_size[2]/2, box_size[2]/2))
        axbig.set_xlabel('Сторона x')
        axbig.set_ylabel('Сторона y')
        axbig.set_zlabel('Сторона z')
        axbig.set_box_aspect([box_size[0], box_size[1], box_size[2]])

    else: 
        axbig = fig.add_subplot(gs[:dimension, :2])
        axbig.set(xlim=(-box_size[0]/2, box_size[0]/2), ylim=(-box_size[1]/2, box_size[1]/2))
        axbig.set_aspect(1)
    
    axbig.set_title('Движение {0} частиц. Момент времени t={1:.2f}. Кол-во столкновений {2}'.format(count_of_particles,time, count_of_collisions))
    fig.tight_layout() 
    
    bbox = axbig.get_tightbbox(fig.canvas.get_renderer(), for_layout_only=True)
    width, height = bbox.width, bbox.height

    size=(width*R_particles/box_size[0]*fig.dpi/89)**2
    
    if dimension == 3: scat = axbig.scatter3D(x[:,0], x[:,1], x[:,2],s=size)
    if dimension == 2:  scat = axbig.scatter(x[:,0], x[:,1],s=size)
    if iteration == iteration2: text='Движение {0} частиц. Момент времени t={1:.2f}. Кол-во столкновений {2:.0f}'
    else: text='Движение {0} частиц. Момент времени t={1:.2f}'
    if count_of_particles>200:  bins_count = 30                              # "по-умному" зададим кол-во баров
    elif count_of_particles>=40: bins_count=(count_of_particles)**1/2/np.log2(count_of_particles)*1.6+4
    elif count_of_particles<40: bins_count=(count_of_particles)**1/2/np.log2(count_of_particles)*1.8+9
    
    HIST_BINS=np.linspace(-4,4,round(bins_count))     # отрисовываем начальные бары и желаемое распределение 
    bars=[]
    lines=[]
    KL_time_history=[[] for i in range(dimension)]
    KL_collisions_history = [[] for i in range(dimension)]
    collisions_history = []
    mu = 0   #скорость центра масс у нас равна 0, поэтому мат ожидание 0
    variance = np.sum(v**2)/count_of_particles  # <v^2>-<v>^2,  dim/2 kT = mv^2 /2, sigma = sqrt(kT/m) => sigma = sqrt(v^2 /dim)
    sigma = (variance/dimension)**(1/2)
    
    for i in range(dimension):
        _,_,bar_cont = ax[i,2].hist(v[:,i],HIST_BINS,density=True)  #строим гистограмму и получаем бары 
        bars.append(bar_cont)
        line, =ax[i,2].plot(np.linspace(mu - 3*sigma, mu + 3*sigma, 100), stats.norm.pdf(np.linspace(mu - 3*sigma, mu + 3*sigma, 100), mu, sigma))
        lines.append(line)
        ax[i,2].set_ylim(top=0.6)  # лимит по y
        ax[i,2].set_title('Распределение скоростей {0}. KL = {1:.3f}'.format(['Vx','Vy','Vz'][i],KL[i]))

    while len(plt.get_fignums()) > 0: #запускаем анимацию, пока открыто больше 1 окна
        collisions_per_iteration = 0
        for j in range(ips):
            x, v, c_p_i = iteration(x, v)
            collisions_per_iteration+=c_p_i    
          
        time+=ips*dt

        if collisions_per_iteration !=0:
            count_of_collisions+=collisions_per_iteration
            collisions_history.append(count_of_collisions) 

        for i in range(dimension):
            newbar,_=np.histogram(v[:,i],HIST_BINS,normed=True)  #находим новые бары 

            mu = np.sum(v[:,i])/count_of_particles   #скорость центра масс у нас равна 0, поэтому мат ожидание 0
            variance = np.sum(v**2)/count_of_particles  # <v^2>-<v>^2,  kT = mv^2 /2, sigma = sqrt(kT/m) => sigma = sqrt(v^2 /2)
            sigma = (variance/2)**(1/2)
            lines[i].set_data(np.linspace(mu - 3*sigma, mu + 3*sigma, 100), stats.norm.pdf(np.linspace(mu - 3*sigma, mu + 3*sigma, 100), mu, sigma))  # из-за того, что потенциальная энергия переходит в кинетическую,успредненный квдарат скорости меняется => меняется распределение
            KL[i] = KLdivergence(mu, sigma, newbar)              #Считаем KL
            KL_time_history[i].append(KL[i])
            if collisions_per_iteration !=0: 
                KL_collisions_history[i].append(KL[i])
            for count, rect in zip(newbar, bars[i].patches):     
              rect.set_height(count)                             #меняем высоту баров
        
        for i in range(dimension): ax[i,2].set_title('Распределение скоростей {0}. KL = {1:.3f}'.format(['Vx','Vy','Vz'][i],KL[i]))
        axbig.set_title(text.format(count_of_particles,time, count_of_collisions))
    
        if dimension==2: scat.set_offsets(np.c_[x[:,0], x[:,1]])
        if dimension==3: scat._offsets3d = (x[:,0], x[:,1], x[:,2])

        plt.draw()                         #отображаем новые данные 
        plt.gcf().canvas.flush_events()

    plt.ioff()
    plt.show()

    return KL_time_history, KL_collisions_history,collisions_history

In [16]:
def noanim(x, v, iteration, rng=100, ips = 10): # Сделано для набора статистики
    time = 0 #время с момента начала анимации, те время движения молекул
    count_of_collisions = 0
    KL = np.zeros(dimension)

    if count_of_particles>200:  bins_count = 30                              # "по-умному" зададим кол-во баров
    elif count_of_particles>=40: bins_count=(count_of_particles)**1/2/np.log2(count_of_particles)*1.6+3
    elif count_of_particles<40: bins_count=(count_of_particles)**1/2/np.log2(count_of_particles)*1.5+7
    HIST_BINS=np.linspace(-4,4,round(bins_count))     # отрисовываем начальные бары и желаемое распределение

    KL_time_history=[[] for i in range(dimension)]
    KL_collisions_history = [[] for i in range(dimension)]
    collisions_history = []

    mu = 0   #скорость центра масс у нас равна 0, поэтому мат ожидание 0
    variance = np.sum(v**2)/count_of_particles  # <v^2>-<v>^2, dimenssion/2 kT = mv^2 /2, sigma = sqrt(kT/m) => sigma = sqrt(v^2 /2)
    sigma = (variance/dimension)**(1/2)
    for j in tqdm(range(rng)): #!!! СЮДА ПОДСТАВИТЬ ВРЕМЯ
        collisions_per_iteration = 0
        for j in range(ips):
            x, v, c_p_i = iteration(x, v)
            collisions_per_iteration+=c_p_i    
          

        count_of_collisions+=collisions_per_iteration
        collisions_history.append(count_of_collisions) 

        time+=ips*dt
        for i in range(dimension):
            newbar,_=np.histogram(v[:,i],HIST_BINS,normed=True)  #находим новые бары 

            mu = np.sum(v[:,i])/count_of_particles   #скорость центра масс у нас равна 0, поэтому мат ожидание 0
            variance = np.sum(v**2)/count_of_particles  # <v^2>-<v>^2, dim/2 kT = mv^2 /2, sigma = sqrt(kT/m) => sigma = sqrt(v^2 /dim)
            sigma = (variance/dimension)**(1/2)
                          #Считаем KL
            KL[i] = KLdivergence(mu, sigma, newbar)
            KL_time_history[i].append(KL[i])
            KL_collisions_history[i].append(KL[i])
    return np.array(KL_time_history), KL_collisions_history,collisions_history

In [17]:
def stat(iteration, count=1,rng =100,ips=10): # кол-во генираций, по которому усредняем, кол-во итераций в течении которого смотрим, ips 
    
    KL_mid = np.zeros((dimension, rng))
    CL_mid = np.zeros((dimension, rng))
    collisions = np.zeros(rng)

    for k in range(count):
        x, v = generate(generation_step=0.2)
        KL_time_history_mid, KL_collisions_history,collisions_history = noanim(x, v, iteration, rng=rng, ips=ips) 
        CL_mid += KL_collisions_history
        KL_mid+=KL_time_history_mid
        collisions+=collisions_history

     
        
    KL_mid/= count
    collisions/=count
    CL_mid/=count
    KL_time_history = [dt*ips*i for i in range(rng)]
    return np.array(KL_mid),  np.array(KL_time_history), np.array(CL_mid), np.array(collisions)

### Основная программа

In [18]:
box_size=np.array([20,20,20])
box_form='parallelepiped'   #если бокс - сфера, то в качестве радиуса будет использовано a_box, в случае размерности 2, размер с_box 'обрубится'
count_of_particles=1000
dimension=3
if dimension == 2 : n_neighborsLJ = int(count_of_particles/(box_size[0]*box_size[0])*10)
if dimension == 3 : n_neighborsLJ = int(count_of_particles/(box_size[0]*box_size[1]*box_size[2])*30)

In [19]:
D1=0.1
triple_collisions = True

In [20]:
x, v = generate(generation_step=0.6)
_,_,_ = animationND(x,v,iteration2,ips=5)

In [21]:
#rng = 3000 # Время эксперимента определяется , как dt*rng*ips
#ips = 10
#print(rng*ips*dt)
#D1=0.1
#KL_time_history_mid1,  KL_time_history,KL_collisions_history,collisions_history = stat(iteration=iteration2, count = 20, rng= rng, ips = ips)

34.479570324217065


  2%|▏         | 62/3000 [00:09<07:16,  6.74it/s]


KeyboardInterrupt: 